In [73]:
import plotly
import plotly.express as px
import json
import numpy as np

In [74]:
with open('output.json') as f:
    data = json.load(f)

In [75]:
def aggregate(data, keep_keys):
    agg = {}
    for data_ptr in data:
        key = tuple([data_ptr[key] for key in keep_keys])
        percent_err = data_ptr['log_err']
        if key not in agg:
            agg[key] = np.array([percent_err, 1])
        else:
            agg[key] += np.array([percent_err, 1])
    keys = list(agg.keys())
    vals = [agg[key][0]/agg[key][1] for key in keys]
    keys = [[key[i] for key in keys] for i in range(len(keep_keys))]
    return *keys, vals

In [76]:
for data_ptr in data:
    data_ptr['err_end'] = abs(np.log(data_ptr["sim_end_time"]) -  np.log(data_ptr["analytic_time"])) / (data_ptr["sim_end_time"]+1e-9) * 100
    data_ptr['err_fit'] = abs(np.log(data_ptr["sim_fit_time"]) -  np.log(data_ptr["analytic_time"])) / (data_ptr["sim_fit_time"]+1e-9) * 100
    data_ptr['log_sim_end_time'] = np.log(data_ptr["sim_end_time"])
    data_ptr['log_sim_fit_time'] = np.log(data_ptr["sim_fit_time"])
    data_ptr['log_analytic_time'] = np.log(data_ptr["analytic_time"])
    data_ptr['log_err_end'] = abs((data_ptr['log_sim_end_time'] - data_ptr['log_analytic_time']) / (data_ptr['log_sim_end_time']+1e-9)) * 100
    data_ptr['log_err_fit'] = abs((data_ptr['log_sim_fit_time'] - data_ptr['log_analytic_time']) / (data_ptr['log_sim_fit_time']+1e-9)) * 100

In [77]:
import pandas as pd
import plotly.graph_objects as go
df = pd.DataFrame(data)
fig = go.Figure(data=[
    go.Scatter3d(x=df["g"], y=df["lambda"], z=df["log_sim_end_time"], mode="markers", marker=dict(size=3)),
    go.Scatter3d(x=df["g"], y=df["lambda"], z=df["log_sim_fit_time"], mode="markers", marker=dict(size=3)),
    go.Scatter3d(x=df["g"], y=df["lambda"], z=df["log_analytic_time"], mode="markers", marker=dict(size=3)),])
fig.show()


In [81]:
df = pd.DataFrame(data)
px.line(df, x="g", y="log_err_fit")

In [79]:
fig = go.Figure(data=[
    go.Line(x=df["g"], y=df["log_sim_fit_time"],  marker=dict(size=3)),
    go.Line(x=df["g"], y=df["log_analytic_time"],  marker=dict(size=3)),])
fig.show()

/Users/yh_huang/venv/generic/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [57]:
gs, lambdas, errs = aggregate(data, ["g", "lambda"])
errs = [err if err<100 else 100 for err in errs]
plot_data = {"g": gs, "lambda": lambdas, "error": errs}
fig = px.scatter_3d(plot_data, x="g", y="lambda", z="error", color="error", title="Error vs g and lambda")
fig.update_traces(marker=dict(size=2))

In [58]:
bs, lambdas, errs = aggregate(data, ["b", "lambda"])
errs = [err if err<100 else 100 for err in errs]
plot_data = {"b": bs, "lambda": lambdas, "error": errs}
fig = px.scatter_3d(plot_data, x="b", y="lambda", z="error", color="error", title="Error vs b and lambda")
fig.update_traces(marker=dict(size=2))

In [59]:
gs, bs, errs = aggregate(data, ["g", "b"])
errs = [err if err<100 else 100 for err in errs]
plot_data = {"g": gs, "b": bs, "error": errs}
fig = px.scatter_3d(plot_data, x="g", y="b", z="error", color="error", title="Error vs g and b")
fig.update_traces(marker=dict(size=2))